# 🚀 Semantic Intelligence: Building a PDF Vector Brain
### **Powered by Mohammad Sefidgar**

Welcome to a masterclass in modern AI retrieval. This notebook transforms static PDF documents into a high-performance, semantically aware vector database. We utilize the power of **Hugging Face**, **LangChain**, and **FAISS** to create a system that doesn't just look for keywords—it understands meaning.

![Diagram](https://github.com/mhsefidgar/AI-Engineering-Pro/blob/main/Practical%20RAG/Semantic%20Search%20AMAZON%20Titan%20Embedding%20FAISS/data/build_pdf_vector_db.jpg?raw=1)

## 🛠️ The Power Stack
To run this engine, you'll need the following tools in your environment:

* **Hugging Face**: Local embeddings for semantic understanding without API keys.
* **LangChain**: The orchestrator for our LLM and vector workflows.
* **FAISS**: Facebook AI Similarity Search, our high-speed vector engine.
* **PyPDF**: To unlock and read PDF data.
* **SemanticChunker**: Part of LangChain Experimental for meaning-based splitting.

In [2]:
# Install local processing requirements
!pip install -qU langchain-huggingface sentence-transformers
!pip install -qU langchain-community pypdf faiss-cpu
!pip install -qU langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 4.6 MB/s eta 0:00:00


In [3]:
!pip install semantic-chunker-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.2/457.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.12.0
    Uninstalling tiktoken-0.12.0:
      Successfully uninstalled tiktoken-0.12.0
  Attempting uninstall: openai
    Found existing ins

In [ ]:
%%capture
import os
import numpy as np
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

# Powering our intelligence with Local Hugging Face Embeddings
# 'all-MiniLM-L6-v2' is fast, accurate, and runs locally without any credentials.
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

print("✅ Local Hugging Face Embeddings initialized.")

## 📂 2. Interactive PDF Upload
Instead of using hardcoded paths, this section allows you to process any custom document on the fly. Simply provide the path to your PDF file.

In [ ]:
from google.colab import files
import os

uploaded = files.upload()

if not uploaded:
    print("❌ No file uploaded.")
else:
    custom_pdf_path = list(uploaded.keys())[0]
    if os.path.exists(custom_pdf_path):
        print(f"📖 Successfully uploaded and located: {custom_pdf_path}")

Saving latestv1 Mohammad Sefidgar DataScience.pdf to latestv1 Mohammad Sefidgar DataScience.pdf
📖 Successfully uploaded and located: latestv1 Mohammad Sefidgar DataScience.pdf


## 🧠 3. Advanced Document Processing
We can process the document using two methods: Traditional Recursive splitting or Advanced Semantic splitting.

In [ ]:
def process_document(file_path, method="semantic"):
    loader = PyPDFLoader(file_path)

    if method == "recursive":
        # Traditional high-speed splitting
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        docs = loader.load_and_split(splitter)
    else:
        # Advanced meaning-based splitting using local embeddings
        splitter = SemanticChunker(embeddings_model, breakpoint_threshold_amount=80)
        raw_docs = loader.load()
        docs = splitter.split_documents(raw_docs)

    # Clean up empty fragments
    clean_docs = [doc for doc in docs if len(doc.page_content) > 0]
    return clean_docs

# Applying Mohammad Sefidgar's semantic logic
processed_docs = process_document(custom_pdf_path, method="semantic")
print(f"✨ Created {len(processed_docs)} semantically coherent chunks.")

✨ Created 10 semantically coherent chunks.


## 🏗️ 4. Vector Database Construction
Injecting our semantically processed documents into FAISS for lightning-fast retrieval.

In [ ]:
vector_db = FAISS.from_documents(processed_docs, embeddings_model)
print(f"🏗️ Vector database created with {vector_db.index.ntotal} vectors.")

🏗️ Vector database created with 10 vectors.


## 🔍 5. Precision Semantic Retrieval
Testing the brain's ability to find relevant information based on meaning.

In [ ]:
query = "What are the key findings or main topics in this document?"
results = vector_db.similarity_search(query, k=3)

print(f"\n🔍 Query: {query}\n")
for i, res in enumerate(results):
    print(f"[Result {i+1}]: {res.page_content[:200]}... [{res.metadata}]\n")


🔍 Query: What are the key findings or main topics in this document?

[Result 1]: • Making reports and documentation for the developed system.... [{'producer': 'Microsoft® Word for Office 365', 'creator': 'Microsoft® Word for Office 365', 'creationdate': '2026-01-06T10:04:30-05:00', 'title': 'Microsoft Word - mh-cv-computervision.docx', 'author': 'Mohammad Sefidgar', 'moddate': '2026-01-06T10:04:30-05:00', 'source': 'latestv1 Mohammad Sefidgar DataScience.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}]

[Result 2]: CGPA.... [{'producer': 'Microsoft® Word for Office 365', 'creator': 'Microsoft® Word for Office 365', 'creationdate': '2026-01-06T10:04:30-05:00', 'title': 'Microsoft Word - mh-cv-computervision.docx', 'author': 'Mohammad Sefidgar', 'moddate': '2026-01-06T10:04:30-05:00', 'source': 'latestv1 Mohammad Sefidgar DataScience.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}]

[Result 3]: Research and Development Engineer | Mehr-Sanat, Iran  
• Research, design, and de

## 💾 6. Local Persistence & Management
Save the vector store locally to avoid re-processing in future sessions.

In [ ]:
db_folder = "custom_pdf_index"
vector_db.save_local(db_folder)
print(f"💾 Vector index successfully saved to {db_folder}")

# Loading the database back
new_db = FAISS.load_local(db_folder, embeddings_model, allow_dangerous_deserialization=True)

print(f"Loaded database contains {new_db.index.ntotal} records.")

💾 Vector index successfully saved to custom_pdf_index
Loaded database contains 10 records.
